### Benchmark model 

First we select the 10 top features

In [33]:
import pandas as pd
from helper_functions import *

In [83]:
importlib.reload(helper_functions)

<module 'helper_functions' from 'C:\\Source\\predicting-house-prices\\helper_functions.py'>

In [3]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [10]:
# This time we train on the complete set
X = train_df.drop('SalePrice', axis=1)
y = train_df['SalePrice']

# We remove categorical features first
X = X.select_dtypes(exclude=['object'])

# Remove columns with null values
X = X.drop(X.columns[X.isnull().any(axis=0)], axis=1)

In [11]:
from sklearn.feature_selection import SelectKBest
sel = SelectKBest(k=10)
sel.fit(X, y)

SelectKBest(k=10, score_func=<function f_classif at 0x0000025AEDCD0D90>)

In [12]:
feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': sel.scores_})

In [14]:
feature_scores.sort_values('Score', ascending=False).head(10)

,Feature,Score
3,OverallQual,5.729183
31,MiscVal,3.756250
14,GrLivArea,3.435110
2,LotArea,3.285852
23,GarageCars,2.861594
17,FullBath,2.678847
24,GarageArea,2.583341
11,1stFlrSF,2.338620
10,TotalBsmtSF,2.315290
5,YearBuilt,2.310198


In [17]:
top_columns = feature_scores.sort_values('Score', ascending=False).head(10)['Feature']

In [20]:
X[top_columns].head()

,OverallQual,MiscVal,GrLivArea,LotArea,GarageCars,FullBath,GarageArea,1stFlrSF,TotalBsmtSF,YearBuilt
0,7,0,1710,8450,2,2,548,856,856,2003
1,6,0,1262,9600,2,2,460,1262,1262,1976
2,7,0,1786,11250,2,2,608,920,920,2001
3,7,0,1717,9550,3,1,642,961,756,1915
4,8,0,2198,14260,3,2,836,1145,1145,2000


We now train and evaluat score on a split

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X[top_columns], y, test_size=.3, random_state=42)

In [48]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression(normalize=True)
reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [49]:
y_hat = reg.predict(X_test)
print_benchmark(y_test, y_hat)

R2-score: 0.798689550915
RMSE (log): 0.2304130665075556


And now train on the whole set provided

In [50]:
reg = LinearRegression(normalize=True)
reg.fit(X[top_columns], y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [51]:
y_hat = reg.predict(X[top_columns])

In [84]:
print_benchmark(y, y_hat)

R2-score: 0.775657233915


C:\Source\predicting-house-prices\helper_functions.py:8: UserWarning: Negative price found in prediction! ¯\_(ツ)_/¯
  warnings.warn('Negative price found in prediction! ¯\_(ツ)_/¯', UserWarning)
C:\Source\predicting-house-prices\helper_functions.py:9: RuntimeWarning: invalid value encountered in log
  return math.sqrt(mean_squared_error(np.log(y_pred), np.log(y_actual)))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#write_submission(test_df, predictions)